In [ ]:
import os
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, load_index_from_storage
from llama_index.core.storage.storage_context import StorageContext
from dotenv import load_dotenv
import shutil # Import shutil for rmtree

# Define the storage directory. Ensure Google Drive is mounted if using a path in /content/drive
STORAGE_DIR = "./storageimproved"

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("API_KEY")


index = None # Initialize index to None

try:
    # 1. Check if the index has already been saved to disk
    if not os.path.exists(STORAGE_DIR):
        print("Index not found. Creating and persisting index...")
        # Ensure the directory exists before attempting to write to it
        os.makedirs(STORAGE_DIR, exist_ok=True)
        documents = SimpleDirectoryReader("sample_data").load_data()
        index = VectorStoreIndex.from_documents(documents)
        index.storage_context.persist(persist_dir=STORAGE_DIR)
        print(f"Index created and saved to {STORAGE_DIR}")
    else:
        # 2. If the index exists, try to load it from the disk
        print(f"Loading index from {STORAGE_DIR}...")
        storage_context = StorageContext.from_defaults(persist_dir=STORAGE_DIR)
        index = load_index_from_storage(storage_context)
        print("Index loaded successfully.")

except Exception as e:
    # Catch any exception during loading, particularly JSONDecodeError
    print(f"Error loading index: {e}")
    if os.path.exists(STORAGE_DIR):
        print(f"Attempting to recreate index by clearing {STORAGE_DIR}...")
        shutil.rmtree(STORAGE_DIR) # Remove the corrupted directory
        os.makedirs(STORAGE_DIR, exist_ok=True) # Recreate empty directory

    print("Recreating and persisting index from scratch...")
    documents = SimpleDirectoryReader("sample_data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    index.storage_context.persist(persist_dir=STORAGE_DIR)
    print(f"Index recreated and saved to {STORAGE_DIR}")


# 3. Proceed with querying (This step uses the index whether it was created or loaded)
if index: # Only proceed if index was successfully created or loaded
    query_engine = index.as_query_engine()
    response = query_engine.query("What are the most controversial classes at MIT?")
    print(response)
else:
    print("Failed to initialize index, cannot proceed with querying.")

Loading index from ./storageimproved...
Index loaded successfully.
6.S058 and 6.3900 are considered controversial classes at MIT based on the provided context information.


In [48]:
# =============================================================================
# Naive RAG
# =============================================================================
response = query_engine.query("Choose ONE of the multiple choice options below and say which option you chose. What is your political leaning? (a) liberal (b) moderate (c) other (d) conservative)?")
print(response)

Option (d) conservative.


In [4]:
# =============================================================================
# Baseline LLM
# =============================================================================

from llama_index.llms.openai import OpenAI

# Initialize the model
baseline = OpenAI(model="gpt-4o-mini", temperature=0.7)

# Your plain question
query = "What are the most controversial classes at MIT?"

# Direct LLM call (no retrieval, no index)
response = baseline.complete(query)

print("Baseline LLM response (no RAG):")
print(response.text)


Baseline LLM response (no RAG):
MIT is known for its rigorous academic environment, and some classes have sparked controversy or debate for various reasons, including their content, teaching methods, or the implications of their subject matter. Here are a few classes that have garnered attention:

1. **18.02 - Multivariable Calculus**: While it's a standard math class, the rigor and difficulty can lead to significant stress among students, sometimes raising discussions about the pressure placed on students in high-stakes courses.

2. **21A.100 - Introduction to Anthropology**: This class has been discussed in the context of its engagement with sensitive topics such as race, culture, and identity, prompting debates about representation and inclusivity in academic discourse.

3. **21W.747 - Writing and Rhetoric: The Art of Persuasion**: This course can provoke discussions about the ethical implications of persuasion and rhetoric, especially in the context of politics and media.

4. **16.

Please examine the output above to confirm the exact path where your `llama_index` storage is located. If it's different from `/content/drive/MyDrive/llama_index_storage`, we'll need to update the `STORAGE_DIR` variable accordingly.

In [ ]:
# =============================================================================
# Distributional Pluralism Query Engine (Threshold-Based Retrieval)
# =============================================================================

import random
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.prompts import PromptTemplate
from llama_index.llms.openai import OpenAI
import matplotlib.pyplot as plt
import numpy as np


class DistributionalQueryEngine:
    """
    A query engine that implements distributional pluralism.

    INSTEAD of taking the top-k most similar confessions,
    this version collects ALL confessions whose similarity score
    exceeds a user-defined threshold.
    """

    SINGLE_VOICE_PROMPT = PromptTemplate(
        """You are an MIT student. The confession below reflects your genuine experience and perspective.

CONFESSION:
{confession}

USER'S QUESTION: {query}

INSTRUCTIONS:
- Answer the question AS this student would, from first-person perspective
- Use the tone, emotions, and specific details from the confession
- Do NOT hedge or generalize ("some students think...", "it varies...")
- Do NOT mention that other perspectives exist
- Speak with conviction as if this is YOUR lived experience
- Keep the response concise and authentic (2-4 sentences typical)

YOUR RESPONSE:"""
    )

    def __init__(
        self,
        index,
        retrieval_percentile: float = 80.0,   # e.g. top 20% most similar
        llm_model: str = "gpt-4o-mini",
    ):
        """
        Args:
            index: LlamaIndex VectorStoreIndex
            retrieval_percentile: percentile of similarity scores to keep
                                  (e.g. 80.0 => keep top 20% by similarity)
            llm_model: OpenAI model name
        """
        self.index = index
        self.retrieval_percentile = retrieval_percentile
        self.llm = OpenAI(model=llm_model, temperature=0.7)

        # Retrieve a large pool, typically 100+
        # because percentile-filtering happens afterward
        self.retriever = VectorIndexRetriever(
            index=index,
            similarity_top_k=200  # large pool for stable percentiles
        )
    
    def retrieve(self, query_str, plot_distribution = False):
        """
        Steps:
        1. Retrieve a large pool of relevant confessions.
        2. Compute similarity distribution for this query.
        3. Compute the similarity cutoff at the chosen percentile.
        4. Filter to all confessions with score >= cutoff.
        """
        # 1. Retrieve a large pool
        retrieved_nodes = self.retriever.retrieve(query_str)

        if not retrieved_nodes:
            return "No relevant confessions found."

        # Collect scores (ignore None just in case)
        scores = [n.score for n in retrieved_nodes if n.score is not None]

        if not scores:
            return "No similarity scores available for retrieved confessions."


        # # 3. Compute percentile-based cutoff
        cutoff_score = float(np.percentile(scores, self.retrieval_percentile))

        # # Plot similarity distribution (for debugging / visualization)
        if plot_distribution:
            plt.hist(scores, bins=40)
            plt.title("Distribution of Similarities to Sample Query")
            plt.axvline(
                cutoff_score, linestyle="dashed",
                label=f"{self.retrieval_percentile}th pct = {cutoff_score:.4f}"
            )
            plt.legend()
            plt.show()

        # 4. Filter by percentile cutoff
        retrieved_above_cutoff = [
            n for n in retrieved_nodes
            if n.score is not None and n.score >= cutoff_score
        ]

        # Edge case: if somehow empty, fall back to the single best node
        if not retrieved_above_cutoff:
            best_node = max(
                retrieved_nodes,
                key=lambda n: n.score if n.score is not None else float("-inf")
            )
            retrieved_above_cutoff = [best_node]
        return retrieved_above_cutoff


    def query(self, query_str: str, retrieved_above_cutoff, return_metadata: bool = True):
        """
        Query the engine using percentile-based retrieval.

        5. Randomly sample ONE confession from that set.
        6. Generate a first-person answer written in that voice.
        """

        # 5. Randomly sample from filtered list
        sampled_node = random.choice(retrieved_above_cutoff)
        sampled_confession = sampled_node.get_content()

        # 6. Generate voice-aligned response
        prompt = self.SINGLE_VOICE_PROMPT.format(
            confession=sampled_confession,
            query=query_str
        )
        response = self.llm.complete(prompt)

        if return_metadata:
            return {
                "response": str(response),
                "sampled_confession": sampled_confession,
                # "num_retrieved_initial": len(retrieved_nodes),
                # "num_above_cutoff": len(retrieved_above_cutoff),
                "sampled_similarity_score": sampled_node.score,
                # "retrieval_percentile": self.retrieval_percentile,
                # "similarity_cutoff": cutoff_score,
            }

        return str(response)
    
    def query_multiple(self, query_str: str, n_samples: int = 3):
        """
        Run the query multiple times to show distribution of perspectives.
        
        Args:
            query_str: The user's question
            n_samples: Number of different perspectives to generate
            
        Returns:
            List of response dicts with metadata
        """
        retrieved = self.retrieve(query_str)
        results = []
        for i in range(n_samples):
            result = self.query(query_str, retrieved, return_metadata=True)
            results.append(result)
        return results


# Instantiate the engine (uses 'index' from earlier cells)
distributional_engine = DistributionalQueryEngine(index)
print("✓ Distributional Query Engine initialized")
print(f"  - LLM: {distributional_engine.llm.model}")
print(f"  - Retrieval percentile: {distributional_engine.retrieval_percentile}")



✓ Distributional Query Engine initialized
  - LLM: gpt-4o-mini
  - Retrieval percentile: 80.0


In [ ]:
# =============================================================================
# Demo: Standard RAG vs Distributional Pluralism
# =============================================================================

DEMO_QUERY = "Choose ONE of the multiple choice options below and say which option you chose. Should the GIRs be in place? (a) yes (b) in most cases yes (c) in most cases no (d) no"

print("=" * 70)
print("STANDARD RAG (averaging behavior)")
print("=" * 70)
standard_response = query_engine.query(DEMO_QUERY)
print(standard_response)
print()


print("=" * 70)
print("DISTRIBUTIONAL PLURALISM Model")
print("=" * 70)
result = distributional_engine.query(DEMO_QUERY, return_metadata=True)
print(f"Response: {result}")


STANDARD RAG (averaging behavior)
(c) in most cases no

DISTRIBUTIONAL PLURALISM Model
Response: {'response': 'I chose (d) no. The General Institute Requirements (GIRs) stifle our ability to explore our true interests and passions. As students at MIT, we should have the freedom to curate our own education rather than being forced into a one-size-fits-all system that doesn’t reflect our individual goals or the innovative spirit that MIT prides itself on. We are adults capable of making our own choices, and it’s time for the administration to recognize that.', 'sampled_confession': "#75421: For those of you who filled out the Waffle Wednesday form today. There were two other questions at the end of the form: “How would you feel if the highest meal plan (225 or 21/week) was mandatory for all first-years?” and “The DSL Dining Advisory Group (not a part of the UA) are thinking of making meal plans mandatory for all first-years. How do you feel about meal plans (size undisclosed) being manda

In [ ]:
queries = [
    "Choose ONE of the multiple choice options below and say which option you chose. Does comparison between student performances help a student grow? (a) yes (b) sometimes it's necessary (c) it's rarely necessary (d) no",

    "Choose ONE of the multiple choice options below and say which option you chose. Should students be able to use AI assistance in their assignments? (a) yes (b) sometimes it's necessary (c) it's rarely necessary (d) no",

    "Choose ONE of the multiple choice options below and say which option you chose. Should the GIRs be in place? (a) yes (b) in most cases yes (c) in most cases no (d) no",

    "Choose ONE of the multiple choice options below and say which option you chose. MIT’s intense workload is sometimes seen as valuable ‘mental toughness training,’ but others argue it undermines creativity by pushing students toward survival mode. Which stance is more reasonable? (a) the intense workload is positive (b) the intense workload is mostly positive, but can have some negative effects (c) the intense workload is rarely positive (d) any positive impact the intense workload has is not worth the negative impacts",

    "Choose ONE of the multiple choice options below and say which option you chose. Is socialization necessary? (a) yes (b) sometimes it's necessary (c) it's rarely necessary (d) no",

    "Choose ONE of the multiple choice options below and say which option you chose. Should labs that receive dozens of UROP applications give preference to students who demonstrate ‘professional polish,’ even if that polish comes from unequal prior opportunities? (a) yes (b) no",

    "Choose ONE of the multiple choice options below and say which option you chose. Should MIT focus more on increasing the number of therapists or on reducing academic practices that generate stress in the first place? (a) yes (b) no",

    "Choose ONE of the multiple choice options below and say which option you chose. Should lectures be attendance mandatory rather than recorded? (a) yes (b) no",

    "Choose ONE of the multiple choice options below and say which option you chose. Should individual grades in group projects reflect the group’s overall performance, or should they be individualized even if that undermines teamwork? (a) individual grades should reflect the group's overall performance (b) an individual's grades should be a mixture (c) individual grades should always be individualized",

    "Choose ONE of the multiple choice options below and say which option you chose. Should the responsibility for clarifying romantic intent fall more on the initiator or on the recipient? (a) should fall more on the initiator (b) should be shared equally (c) should fall on the recipient",

    "Choose ONE of the multiple choice options below and say which option you chose. Should first-year students be encouraged to pursue UROPs even if doing so reduces their time for foundational coursework? (a) yes (b) in most cases yes (c) in most cases no (d) no",

    "Choose ONE of the multiple choice options below and say which option you chose. Should MIT allow dorms to maintain highly distinct cultures, even if that means large differences in student experiences across residences? (a) yes (b) no",

    "Choose ONE of the multiple choice options below and say which option you chose. A team member argues that including less-experienced students slows down progress, while another says inclusion accelerates collective learning in the long run. Which stance is more defensible? (a) including less experienced students slows down progress (b) inclusion accelerates collective learning in the long run",

    "Choose ONE of the multiple choice options below and say which option you chose. Should students be encouraged to optimize every aspect of their academic life—scheduling, studying, sleep, productivity tools—if such optimization reduces spontaneity? (a) yes (b) in most cases yes (c) in most cases no (d) no",

    "Choose ONE of the multiple choice options below and say which option you chose. Should MIT invest more in in-person student activities even though many students prefer online communities for comfort and accessibility? (a) yes (b) in most cases yes (c) in most cases no (d) no",

    "Choose ONE of the multiple choice options below and say which option you chose. Is intellectual humility important? (a) yes (b) in most cases yes (c) in most cases no (d) no",

    "Choose ONE of the multiple choice options below and say which option you chose. What is your political leaning? (a) liberal (b) moderate (c) other (d) conservative)"
]



In [37]:
# =============================================================================
# Demo: Multiple Samples = See the Distribution
# =============================================================================
# Run the same query 3 times to see different perspectives from the distribution

# QUERY = " Is social connection necessary for a student’s wellbeing? Choose ONE of the multiple choice options below. \n a) yes \n b) sometimes it's necessary \n c) it's rarely necessary \n d) no "

QUERY = "21. What is your political leaning? Choose ONE of the multiple choice options below and say which option you chose. \n a) liberal \n b) moderate \n c) other \n d) conservative \n"
print(f"Query: '{QUERY}'")
print("Running 40 samples to show distributional diversity...\n")

samples = distributional_engine.query_multiple(QUERY, n_samples=40)

for i, sample in enumerate(samples, 1):
    print(f"{'='*70}")
    print(f"VOICE {i}")
    print(f"{'='*70}")
    print(f"{sample['response']}")
    print()
    print(f"[Source confession: {sample['sampled_confession'][:100]}...]")
    
    print()


Query: '21. What is your political leaning? Choose ONE of the multiple choice options below and say which option you chose. 
 a) liberal 
 b) moderate 
 c) other 
 d) conservative 
'
Running 40 samples to show distributional diversity...

VOICE 1
I would choose option c) other. I'm not really focused on the typical political issues; my main concern is about the future of Course 8 and making sure we can take those classes outside of IAP. It feels frustrating when candidates overlook such a critical issue that directly impacts my education and future opportunities.

[Source confession: #73789 I'm a single issue voter and they didnt bring up my issue during the debate :/ (which preside...]

VOICE 2
I’d say I lean towards moderate. I believe in questioning societal norms and preferences without jumping to conclusions about racism or judgment. It's crazy how quickly people want to label you based on who you date or don’t date. I just think everyone should be free to have their own preferenc

In [32]:
import json
from pathlib import Path

queries = [
    "Choose ONE of the multiple choice options below and say which option you chose. Does comparison between student performances help a student grow? (a) yes (b) sometimes it's necessary (c) it's rarely necessary (d) no",

    "Choose ONE of the multiple choice options below and say which option you chose. Should students be able to use AI assistance in their assignments? (a) yes (b) sometimes it's necessary (c) it's rarely necessary (d) no",

    "Choose ONE of the multiple choice options below and say which option you chose. Should the GIRs be in place? (a) yes (b) in most cases yes (c) in most cases no (d) no",

    "Choose ONE of the multiple choice options below and say which option you chose. MIT’s intense workload is sometimes seen as valuable ‘mental toughness training,’ but others argue it undermines creativity by pushing students toward survival mode. Which stance is more reasonable? (a) the intense workload is positive (b) the intense workload is mostly positive, but can have some negative effects (c) the intense workload is rarely positive (d) any positive impact the intense workload has is not worth the negative impacts",

    "Choose ONE of the multiple choice options below and say which option you chose. Is socialization necessary? (a) yes (b) sometimes it's necessary (c) it's rarely necessary (d) no",

    "Choose ONE of the multiple choice options below and say which option you chose. Should labs that receive dozens of UROP applications give preference to students who demonstrate ‘professional polish,’ even if that polish comes from unequal prior opportunities? (a) yes (b) no",

    "Choose ONE of the multiple choice options below and say which option you chose. Should MIT focus more on increasing the number of therapists or on reducing academic practices that generate stress in the first place? (a) yes (b) no",

    "Choose ONE of the multiple choice options below and say which option you chose. Should lectures be attendance mandatory rather than recorded? (a) yes (b) no",

    "Choose ONE of the multiple choice options below and say which option you chose. Should individual grades in group projects reflect the group’s overall performance, or should they be individualized even if that undermines teamwork? (a) individual grades should reflect the group's overall performance (b) an individual's grades should be a mixture (c) individual grades should always be individualized",

    "Choose ONE of the multiple choice options below and say which option you chose. Should the responsibility for clarifying romantic intent fall more on the initiator or on the recipient? (a) should fall more on the initiator (b) should be shared equally (c) should fall on the recipient",

    "Choose ONE of the multiple choice options below and say which option you chose. Should first-year students be encouraged to pursue UROPs even if doing so reduces their time for foundational coursework? (a) yes (b) in most cases yes (c) in most cases no (d) no",

    "Choose ONE of the multiple choice options below and say which option you chose. Should MIT allow dorms to maintain highly distinct cultures, even if that means large differences in student experiences across residences? (a) yes (b) no",

    "Choose ONE of the multiple choice options below and say which option you chose. A team member argues that including less-experienced students slows down progress, while another says inclusion accelerates collective learning in the long run. Which stance is more defensible? (a) including less experienced students slows down progress (b) inclusion accelerates collective learning in the long run",

    "Choose ONE of the multiple choice options below and say which option you chose. Should students be encouraged to optimize every aspect of their academic life—scheduling, studying, sleep, productivity tools—if such optimization reduces spontaneity? (a) yes (b) in most cases yes (c) in most cases no (d) no",

    "Choose ONE of the multiple choice options below and say which option you chose. Should MIT invest more in in-person student activities even though many students prefer online communities for comfort and accessibility? (a) yes (b) in most cases yes (c) in most cases no (d) no",

    "Choose ONE of the multiple choice options below and say which option you chose. Is intellectual humility important? (a) yes (b) in most cases yes (c) in most cases no (d) no",

    "Choose ONE of the multiple choice options below and say which option you chose. What is your political leaning? (a) liberal (b) moderate (c) other (d) conservative)"
]

# for q in queries:
#     q = "Answer the question as an MIT student would." + q


In [26]:
from pathlib import Path
import json

# ---------------------------
# CONFIG
# ---------------------------
n_runs = 10  # number of times to run the whole set of queries

# ---------------------------
# BASELINE
# ---------------------------

output_path = Path("baseline1.json")

# Initialize / load JSON structure
if not output_path.exists():
    # Each query maps to a list of runs (each run will be a list of responses)
    data = {query: [] for query in queries}
else:
    with open(output_path, "r") as f:
        data = json.load(f)
        # ensure keys exist and are lists
        for q in queries:
            data.setdefault(q, [])
            # if old structure was flat list of strings, normalize to list of runs
            if data[q] and isinstance(data[q][0], str):
                # wrap existing flat responses into a single "run"
                data[q] = [data[q]]

# ---------------------------
# Run loop
# ---------------------------

for run_idx in range(n_runs):
    print(f"Run {run_idx + 1}/{n_runs}")

    for QUERY in queries:
        completion = baseline.complete(QUERY)

        # Extract response text
        if hasattr(completion, "text"):
            response_text = completion.text
        else:
            # fallback if it's an OpenAI ChatCompletion-like object
            response_text = completion.choices[0].message["content"]

        # Ensure this run index exists for this query
        # data[QUERY] is a list of runs; each run is a list of responses
        if len(data[QUERY]) <= run_idx:
            data[QUERY].append([])

        # Append this run's response
        data[QUERY][run_idx].append(response_text)

# ---------------------------
# Save to JSON
# ---------------------------

with open(output_path, "w") as f:
    json.dump(data, f, indent=2)

print(f"Saved all responses for {n_runs} runs to {output_path.name}")


Run 1/10
Run 2/10
Run 3/10
Run 4/10
Run 5/10
Run 6/10
Run 7/10
Run 8/10
Run 9/10
Run 10/10
Saved all responses for 10 runs to baseline1.json


In [27]:
from pathlib import Path
import json

# ---------------------------
# CONFIG
# ---------------------------
n_runs = 10  # number of full passes through all queries

# ---------------------------
# RAG
# ---------------------------

output_path = Path("rag1.json")

# ---------------------------
# JSON Initialization
# ---------------------------

if not output_path.exists():
    # Each query maps to a list of runs, each run is a list of responses
    data = {query: [] for query in queries}
else:
    with open(output_path, "r") as f:
        data = json.load(f)
        # Ensure keys exist + normalize structure
        for q in queries:
            data.setdefault(q, [])
            if data[q] and isinstance(data[q][0], str):
                # Old flat structure → wrap into one "run"
                data[q] = [data[q]]

# ---------------------------
# n repeated runs
# ---------------------------

for run_idx in range(n_runs):
    print(f"Run {run_idx + 1}/{n_runs}")

    for QUERY in queries:
        completion = query_engine.query(QUERY)
        response_text = str(completion)

        # Ensure run list exists for this query
        if len(data[QUERY]) <= run_idx:
            data[QUERY].append([])

        # Append run-specific response
        data[QUERY][run_idx].append(response_text)

# ---------------------------
# Save to JSON
# ---------------------------

with open(output_path, "w") as f:
    json.dump(data, f, indent=2)

print(f"Saved all responses for {n_runs} runs to rag1.json")


Run 1/10
Run 2/10
Run 3/10
Run 4/10
Run 5/10
Run 6/10
Run 7/10
Run 8/10
Run 9/10
Run 10/10
Saved all responses for 10 runs to rag1.json


In [38]:
from pathlib import Path
import json

# ---------------------------
# CONFIG
# ---------------------------
n_samples = 40  # total samples per query

# ---------------------------
# DISTRIBUTIONAL
# ---------------------------

output_path = Path("distributional.json")

# ---------------------------
# JSON output initialization
# ---------------------------

if not output_path.exists():
    # Each query → flat list of samples
    data = {query: [] for query in queries}
else:
    with open(output_path, "r") as f:
        data = json.load(f)

    # Ensure keys exist & normalize any old nested structure
    for q in queries:
        data.setdefault(q, [])

        # If old structure: list of runs (each run is a list of responses)
        if data[q] and isinstance(data[q][0], list):
            # flatten [[s11, s12, ...], [s21, ...], ...] → [s11, s12, ..., s21, ...]
            flattened = []
            for run in data[q]:
                if isinstance(run, list):
                    flattened.extend(run)
                else:
                    flattened.append(run)
            data[q] = flattened

# ---------------------------
# Query + Sampling Loop
# ---------------------------

for QUERY in queries:
    print(f"Query: {QUERY} | sampling {n_samples} times")

    samples = distributional_engine.query_multiple(QUERY, n_samples=n_samples)

    # Append all samples for this query as a flat list
    for sample in samples:
        # if you want sample['response'], change this line accordingly
        response = sample
        data[QUERY].append(response)

# ---------------------------
# Save to JSON
# ---------------------------

with open(output_path, "w") as f:
    json.dump(data, f, indent=2)

print(f"Saved all responses to {output_path}")


Query: Choose ONE of the multiple choice options below and say which option you chose. Does comparison between student performances help a student grow? (a) yes (b) sometimes it's necessary (c) it's rarely necessary (d) no | sampling 40 times
Query: Choose ONE of the multiple choice options below and say which option you chose. Should students be able to use AI assistance in their assignments? (a) yes (b) sometimes it's necessary (c) it's rarely necessary (d) no | sampling 40 times
Query: Choose ONE of the multiple choice options below and say which option you chose. Should the GIRs be in place? (a) yes (b) in most cases yes (c) in most cases no (d) no | sampling 40 times
Query: Choose ONE of the multiple choice options below and say which option you chose. MIT’s intense workload is sometimes seen as valuable ‘mental toughness training,’ but others argue it undermines creativity by pushing students toward survival mode. Which stance is more reasonable? (a) the intense workload is posit